## Logger

In [ ]:
import datetime
import time
import logging
import sys

def set_logger():
  logging.basicConfig(level=logging.INFO)
  logger_t = logging.getLogger('withoutlevel')
  logger = logging.getLogger('withlevel')

  f_handler = logging.FileHandler('Falldetection.log')
  f_handler_t = logging.FileHandler('Falldetection.log')

  f_format = logging.Formatter('%(levelname)s - %(message)s')
  f_format_t = logging.Formatter('%(message)s')

  f_handler.setFormatter(f_format)
  f_handler_t.setFormatter(f_format_t)

  logger.addHandler(f_handler)
  logger_t.addHandler(f_handler_t)
  return logger, logger_t

In [ ]:
import datetime
import time
t_set = lambda: datetime.datetime.now().astimezone()
t_diff = lambda t: str(t_set() - t)
t_stamp = lambda t=None: str(t) if t else str(t_set())
t = t_set()
time.sleep(5)
t_diff(t)

'0:00:05.010771'

In [ ]:
import logging
import sys

#logger without time
logging.basicConfig(level=logging.INFO)
logger_t = logging.getLogger('withoutlevel')
logger = logging.getLogger('withlevel')

f_handler = logging.FileHandler('Falldetection.log')
f_handler_t = logging.FileHandler('Falldetection.log')

f_format = logging.Formatter('%(levelname)s - %(message)s')
f_format_t = logging.Formatter('%(message)s')

f_handler.setFormatter(f_format)
f_handler_t.setFormatter(f_format_t)

logger.addHandler(f_handler)
logger_t.addHandler(f_handler_t)


In [ ]:
import platform
platform.uname()
for det in platform.uname():
  logger_t.info(str(det))

INFO:withoutlevel:Linux
INFO:withoutlevel:186777b194e6
INFO:withoutlevel:5.4.109+
INFO:withoutlevel:#1 SMP Tue Apr 20 19:55:43 PDT 2021
INFO:withoutlevel:x86_64
INFO:withoutlevel:x86_64


In [ ]:
logger_t.info('='*100)
logger_t.info("\nFalldetection project")

INFO:withoutlevel:====================================================================================================
INFO:withoutlevel:
Falldetection project


## Download dataset

In [ ]:
a = 'hello'

In [ ]:
import urllib.request as urlreq
from zipfile import ZipFile
import shutil
import os
from tqdm.notebook import tqdm

logger_t.info(f'{"="*100} \n\nSTEP 1: Download dataset into the system')

def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

def extract_zip(src, dest, map):
    '''
    Extracts the contents downloaded from the URL
    '''
    zip_ref = ZipFile(src,'r')
    for name in zip_ref.namelist():
      ext = name.split('.')[-1]
      if ext in map.keys():
        dest = os.path.join(dest,map[ext])
        break
    zip_ref.extractall(dest)
    logger_t.info(f'... into {dest}')
    zip_ref.close()

def cleanup(dataPath):
  '''
  clean the temp files
  '''
  for folder in os.listdir(dataPath):
    if folder == 'temp':
      cleanup_path = os.path.join(dataPath,'temp')
      shutil.rmtree(cleanup_path)
      break
  return

def download(url, root):
  '''
  downloads the file from the url
  and put it in desired folder
  '''
  
  map = {'csv':'csv', 'zip':'temp', 'png':'frame', 'jpg':'frame', 'mp4':'video', 'avi':'video'}
  ext = url.split('.')[-1] 
  # raise if the url is of not file type
  assert ext in map.keys(), "invalid url"
  name = url.split('/')[-1]
  filePath = os.path.join(root, map[ext])
  safedirs(filePath)
  filename = os.path.join(filePath,name)
  if os.path.exists(filename):
    return 
  logger_t.info(f"Downloading data files from URL:{url}")
  urlreq.urlretrieve(url, filename)
  #if the file downloaded is zip file then extract original files
  if ext == 'zip':
    extract_zip(filename,root,map)
    

def urfd(dataPath,num=40,clean=False):
  '''
  download the files from the URFD data url
  dataPath: destination path
  num: number of files to download
  clean: whether to clean the temp files
  '''
  #csv file url's
  csv = ['http://fenix.univ.rzeszow.pl/~mkepski/ds/data/urfall-cam0-adls.csv',
         'http://fenix.univ.rzeszow.pl/~mkepski/ds/data/urfall-cam0-falls.csv']
  #download csv files
  for f in csv:
    t = t_set()
    download(f,dataPath)
    logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}.\n')
  #download frame zip files from the project website
  for n in tqdm(range(1,num+1)):
    # Add zeros to adapt files in Website
    if n < 10:
      n = '0'+str(n)
    #only 31 Fall sequences available
    if int(n)<31:
      t = t_set()
      fall_url = f'http://fenix.univ.rzeszow.pl/~mkepski/ds/data/fall-{n}-cam0-rgb.zip'
      download(fall_url,datapath)
      logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}\n')
    t=t_set()
    adl_url = f'http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-{n}-cam0-rgb.zip'
    download(adl_url,dataPath)
    logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}\n')
    
    if n == num-1 and clean:
      cleanup(dataPath)




root = os.getcwd()
datapath = root + '/dataset/urfd'
logger.info(f'Dateset is downloaded in path {datapath}')
t= t_set()
urfd(datapath) 
logger.info(f'Total time for downloading and extracting entire dataset is {t_diff(t)}\n {"="*200}')
    

## Label enrichment

In [ ]:
import pandas as pd
import os

logger_t.info(f'{"="*200} \n\nSTEP 2: Assign labels to each frames')

def assign_label(df,root,filenames):
  for filename in filenames:
    #processed files contain '_' hence we can omit those
    if '_' in filename:
      continue
    #get the extension from the file
    ext = filename.split('.')[-1]
    split_array = filename.split('.')[0].split('-')
    #extract id and frame
    file_id,frame_id = '-'.join(split_array[:2]), split_array[-1]
    #adl files doesnt contain falls hence assigning label 0 to all files
    if 'adl' in file_id:
      label = 0
    else:
      #check the label in dataframe for id and frame
      df_label = int(df[(df[0] == file_id) & (df[1] == int(frame_id))][2])
      label = 1 if df_label == 1 else 0
    #Rename the files
    name = f'{file_id}_{frame_id}_{label}.{ext}'
    new_path = os.path.join(root, name)
    old_path = os.path.join(root,filename)
    os.rename(old_path,new_path)
    # create a entry in the dataframe for new path of the frame file
    df.loc[(df[0] == file_id) & (df[1] == int(frame_id)), 'path'] = new_path
    df.loc[(df[0] == file_id) & (df[1] == int(frame_id)), 2] = label


def enrich_labels(df,dataPath,folder=True):
  '''
  Embed labels into the data filenames
  df: data frame consisting of details on labels
  datapath: the path of the folder containing frames
  folder: indicates the structure of the data
  '''
  if folder:
    root, dirs, _ = next(os.walk(dataPath))
    dir_array = [os.path.join(root,dir) for dir in dirs]
    for path in dir_array:
      root, _, files = next(os.walk(path))
      assign_label(df,root,files)
  else:
    root, _, files = next(os.walk(dataPath))
    assign_label(df, root, files)




INFO:withoutlevel:======================================================================================================================================================================================================== 

STEP 2: Assign labels to each frames


In [ ]:
# create dataframe for further processing using the CSV file data
csv = root + '/dataset/urfd/csv'
df_fall = pd.read_csv('/content/dataset/urfd/csv/urfall-cam0-falls.csv',header=None)
df_adl = pd.read_csv('/content/dataset/urfd/csv/urfall-cam0-adls.csv',header=None)
dfs = pd.concat([df_fall,df_adl],ignore_index=True)
dfs = dfs[[0,1,2]]
dfs['path'] = None
logger_t.info(f"Enrich labels into the filenames extractend from the CSV files")
t = t_set()
enrich_labels(dfs,'/content/dataset/urfd/frame')
logger.info(f'Elapsed time for modifying filenames: {t_diff(t)}s.')

INFO:withoutlevel:Enrich labels into the filenames extractend from the CSV files
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:withlevel:Elapsed time for modifying filenames: 0:00:41.846719s.


## Dataset Creation

In [ ]:

import numpy as np
import os
import shutil
from tqdm.notebook import tqdm
import torch
import random
from torch.utils.data import random_split



def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

def createDataset(df,dataPath,destPath,ids,clip_len=50,size=60):
  random.seed(43)
  logger.info(f'Seed value is 43')
  assert size % 60 == 0, "Size should be in the multiple of 60"
  assert type(ids) == list, "expecting a list of strings"
  data_files = {i:[] for i in ids}
  for root,dir,files in os.walk(dataPath):
    for f in files:
      id = f.split('_')[0]
      if id in ids:
        data_files[id].append(os.path.join(root,f))
  for subset in range(size//60):
    t = t_set()
    for idx in range(len(list(data_files.keys()))):
      path = sorted(data_files[list(data_files.keys())[idx]])
      topic = (list(data_files.keys())[idx])
      while True:
        root_path = os.path.join(destPath, topic.split('-')[0], topic+'_'+str(subset))
        safedirs(root_path)
        rand_n = random.randint(0,len(path)-clip_len)
        frame_path = path[rand_n : rand_n+clip_len]
        labels = []
        for fp in frame_path:
          shutil.copy(fp, os.path.join(root_path, os.path.basename(fp)))
          #frame is the datapath with label at the end of the filename
          labels.append(int(fp.split('.')[0][-1]))
        if topic.split('-')[0] == 'adl':
          break
        elif topic.split('-')[0] == 'fall' and labels.count(1) > 4:
          break
        shutil.rmtree(root_path)
    logger.info(f'time elapsed for each subset {subset} is {t_diff(t)}')



In [ ]:
from torch.utils.data import random_split
#120 image dataset
#shutil.rmtree('/content/subset')

logger_t.info(f'{"="*100} \n\nSTEP 3: Create subset of 60/120/180 files of 50 Frames each from the Dataset')

size = 180
logger.info(f'Size of the subset: {size}')
ids = dfs[0].unique()[:60]
train_size = int(0.8*len(ids))
val_size = int(0.8*(len(ids) - train_size))
test_size = len(ids) - (train_size + val_size)
train_ids,valid_ids,test_ids = random_split(ids,[train_size,val_size,test_size],generator=torch.Generator().manual_seed(1))
logger_t.info(f'train_size={train_size}\nvalidation_size={val_size}\ntest_size={test_size}')
logger_t.info(f'train_ids={train_ids}\nvalidation ids={valid_ids}\ntest_ids={test_ids}')

train_destPath = os.path.join(os.getcwd(),'subset/train')
valid_destPath = os.path.join(os.getcwd(),'subset/valid')
test_destPath = os.path.join(os.getcwd(),'subset/test')
logger_t.info(f'train path= {train_destPath}\nvalidation path= {valid_destPath}\nTest path= {test_destPath}')

t = t_set()
createDataset(dfs,'/content/dataset/urfd/frame',train_destPath,ids = list(train_ids),size=size)
logger.info(f"Time elapsed for creating Train subset is {t_diff(t)}")
t = t_set()
createDataset(dfs,'/content/dataset/urfd/frame',valid_destPath,ids = list(valid_ids),size=size)
logger.info(f"Time elapsed for creating Validation subset is {t_diff(t)}")
t = t_set()
createDataset(dfs,'/content/dataset/urfd/frame',test_destPath,ids = list(test_ids),size=size)
logger.info(f"Time elapsed for creating Test subset is {t_diff(t)}")


## BBOX creation

In [ ]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time,os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

def person_detector(image):
  '''
  Moving object detection by frame differencing
  input: torch tensor object of shape [batch,50,3,480,640] and labels array
  output: tuple of torch tensor [batch,1,224,224] and label object 
  '''
  temp_img = image.numpy()
  center_points = []
  for i in range(0,temp_img.shape[0] - 2):
    img1 = cv2.medianBlur(cv2.cvtColor(temp_img[i],cv2.COLOR_BGR2GRAY),5)
    img2 = cv2.medianBlur(cv2.cvtColor(temp_img[i+2],cv2.COLOR_BGR2GRAY),5)
    (score,diff) = structural_similarity(img1,img2,full=True)
    diff = (diff * 255).astype('uint8')
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    dilate_frame = cv2.dilate(thresh, None, iterations=5)
    contours = cv2.findContours(dilate_frame.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1] 
    val = np.argmax([cv2.contourArea(c) for c in contours])
    x,y,w,h = cv2.boundingRect(contours[val])
    center = (int((x+w)/2), int((y+h)/2))
    center_points.append(center)
  return center_points


def get_transform():
  transform = tf.keras.Sequential([
                                   preprocessing.Resizing(224,224),
                                   preprocessing.Normalization()
  ])
  return transform

def image_embedding(images,labels):
  transform = get_transform()
  background = []
  lbls = []
  for image,label in zip(images,labels):
    lbl=[0,0]
    bkg = np.zeros((480,640),np.uint8)
    start = time.time()
    points = person_detector(image) 
    for p in range(len(points)-1):
      bkg = cv2.line(bkg,points[p],points[p+1],(255,0,0),5)
    bkg = transform(tf.expand_dims(bkg,2))
    lbl[1] = 1 if 1 in label else 0
    lbl[0] = 1 if 1 not in label else 0
    background.append(bkg)
    lbls.append(lbl)
  return tf.stack(background,axis=0), tf.convert_to_tensor(lbls,dtype=tf.float16)

transform = get_transform()


In [ ]:
import numpy as np
import cv2
import os
import random
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import math


class clipDataset(Sequence):
  def __init__(self,datapath,mode='train',batch_size=2):
    self.batch_size= batch_size
    self.datapath = datapath
    self.data_dict = defaultdict(list)
    for root,dir,files in os.walk(datapath):
      for f in files:
        frame_path = os.path.join(root,f)
        folder = os.path.basename(root)
        if mode in frame_path:
          self.data_dict[folder].append(frame_path)
  
  def __len__(self):
    return math.ceil(len(self.data_dict)/self.batch_size)
  
  def __getitem__(self,index):
    keys = list(self.data_dict.keys())[index*self.batch_size:(index+1)*self.batch_size]
    frames_stack = []
    labels_stack = []
    for key in keys:
      frames = []
      labels = []
      files_list = self.data_dict[key]
      for f in files_list:
        labels.append(int(f.split('.')[0][-1]))
        #read the image, convert to RGB and then to Tensor
        tensor_img = tf.convert_to_tensor(cv2.cvtColor(cv2.imread(f),cv2.COLOR_BGR2RGB))
        frames.append(tensor_img)
      frames_stack.append(tf.stack(frames,axis=0))
      labels_stack.append(tf.convert_to_tensor(labels,dtype=tf.float16))
    frames_stack = tf.stack(frames_stack,axis=0)
    labels_stack = tf.stack(labels_stack,axis=0)
    frames,labels = image_embedding(frames_stack,labels_stack)
    return frames,labels

valid = clipDataset('/content/subset',mode='valid',batch_size=2)
f,l= valid[0]



## Embedding data

In [ ]:
#shutil.rmtree('/content/boundingBox')
#shutil.rmtree('/content/trajectories')

In [ ]:
from collections import defaultdict
from skimage.metrics import structural_similarity
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time,os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing


def person_detector(imageMatrix,label,mode,folder):
  '''
  Moving object detection by frame differencing
  input: torch tensor object of shape [batch,50,3,480,640] and labels array
  output: tuple of torch tensor [batch,1,224,224] and label object 
  '''
  info = ['adl','fall']
  bbox_path = os.path.join(os.getcwd(), 'boundingBox', mode, info[label], folder)
  logger_t.info(f'Working on creating ... {bbox_path} set')
  safedirs(bbox_path)
  temp_img = imageMatrix
  center_points = []
  for i in range(0,temp_img.shape[0] - 2):
    img1 = cv2.medianBlur(cv2.cvtColor(temp_img[i],cv2.COLOR_BGR2GRAY),5)
    img2 = cv2.medianBlur(cv2.cvtColor(temp_img[i+2],cv2.COLOR_BGR2GRAY),5)
    (score,diff) = structural_similarity(img1,img2,full=True)
    diff = (diff * 255).astype('uint8')
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    dilate_frame = cv2.dilate(thresh, None, iterations=5)
    contours = cv2.findContours(dilate_frame.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1] 
    val = np.argmax([cv2.contourArea(c) for c in contours])
    x,y,w,h = cv2.boundingRect(contours[val])
    cv2.rectangle(img1, (x, y), (x + w, y + h), (36,255,12), 2)
    path = os.path.join(bbox_path, f'image_{i}.jpg')
    cv2.imwrite(path, img1)
    center = (int((x+w)/2), int((y+h)/2))
    center_points.append(center)
  return center_points

def get_transform():
  transform = tf.keras.Sequential([
                                   preprocessing.Resizing(224,224),
                                   preprocessing.Normalization()
  ])
  return transform


def image_embedding(images,label,mode,filename):
  transform = get_transform()
  info = ['adl', 'fall']
  trajectory_path = os.path.join(os.getcwd(), 'trajectories', mode, info[label])
  safedirs(trajectory_path)
  logger_t.info(f'creating trajectory image at {trajectory_path}')
  bkg = np.zeros((480,640),np.uint8)
  t=t_set()
  points = person_detector(images,label,mode,filename) 
  logger.info(f'Elapsed time for Bounding box calculation is {t_diff(t)}')
  for p in range(len(points)-1):
    bkg = cv2.line(bkg,points[p],points[p+1],(255,0,0),5)
  bkg = transform(tf.expand_dims(bkg,2))
  path = os.path.join(trajectory_path, filename+'.jpg')
  cv2.imwrite(path, bkg.numpy())


def embedded_data(datapath):
  data_dict = defaultdict(list)
  for root,dir,files in os.walk(datapath):
      for f in files:
        frame_path = os.path.join(root,f)
        #print(root)
        folder = os.path.basename(root)
        data_dict[folder].append(frame_path)
  for key in list(data_dict.keys()):
    frames, labels = [], []
    for f in sorted(data_dict[key]):
      labels.append(int(f.split('.')[0][-1]))
      temp = f.split('/')
      mode = temp[3]
      filename = temp[5]
      img = cv2.cvtColor(cv2.imread(f),cv2.COLOR_BGR2RGB)
      frames.append(img)
    frames = np.stack(frames,axis=0)
    t=t_set()
    image_embedding(frames, np.max(labels), mode, filename)
    logger.info(f"Total time taken for {key} is {t_diff(t)}")


logger_t.info(f'{"="*100} \n\nSTEP 4: Create Trajectory images\n')
t = t_set()
root = os.getcwd()
embedded_data(root + '/subset')
logger.info(f'Total time taken to create trajectory images: {t_diff(t)}')


INFO:withoutlevel:==================================================================================================== 

STEP 4: Create Trajectory images

INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/adl-11_2 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:01.677573
INFO:withlevel:Total time taken for adl-11_2 is 0:00:02.034584
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/adl-29_1 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:01.593175
INFO:withlevel:Total time taken for adl-29_1 is 0:00:01.615324
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/adl-19_2 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:01.622248

In [ ]:
sys.stdout = open('Falldetection.log','a')

In [ ]:
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Data generators\n')
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
batch_size = 32
logger.info(f'batch size: {batch_size}')
root = os.getcwd()
train = datagen.flow_from_directory(root + '/trajectories/train',target_size=(224, 224),color_mode='grayscale',batch_size=batch_size)
valid = datagen.flow_from_directory(root + '/trajectories/valid',target_size=(224, 224),color_mode='grayscale',batch_size=batch_size)
test = datagen.flow_from_directory(root + '/trajectories/test',target_size=(224, 224),color_mode='grayscale',batch_size=batch_size)

INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Data generators

INFO:withlevel:batch size: 32


In [ ]:
train_gen[0][0].shape

(32, 224, 224, 1)

In [ ]:
train = clipDataset('/content/subset',mode='train',batch_size=2)
valid = clipDataset('/content/subset',mode='valid',batch_size=2)
test = clipDataset('/content/subset',mode='test',batch_size=2)

In [ ]:
train[0][0].shape

TensorShape([2, 224, 224, 1])

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,Dense,Dropout
from tensorflow.keras.metrics import Precision, Recall
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Model and train the model\n')
model = tf.keras.Sequential([
              tf.keras.Input(shape=(224,224,1)),
              Conv2D(32,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              Conv2D(64,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              GlobalAveragePooling2D(),
              Dense(128),
              Dropout(0.5),
              Dense(64),
              Dropout(0.5),
              Dense(2,activation='sigmoid')])

model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', Precision(), Recall()])
logger.info(f'{model.summary()}')

history = model.fit_generator(generator=train,
                              validation_data=valid,
                              epochs=10,
                              verbose=1)


INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Model and train the model

INFO:withlevel:None
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


In [ ]:

logger.info(f'Evaluation Results')
model.evaluate(test)

INFO:withlevel:Evaluation Results


[0.13619157671928406, 1.0, 1.0, 1.0]